In [17]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
np.random.seed(42)

In [18]:
data = pd.read_excel('../Exel/data.xlsx', index_col=None)  

In [19]:
data

,Measurement_number,network_device,CPU_clock_frequency_GHz,UDP_Throughput_Mb/sec,CPU_utilization_%,CPU_temperature_C
0,1,1,1500,600,50,38
1,2,1,1500,650,55,45
2,3,1,1500,700,60,52
3,4,1,1500,750,65,60
4,5,1,1500,900,70,67
5,6,2,2600,750,35,30
6,7,2,2600,800,40,31
7,8,2,2600,850,45,32
8,9,2,2600,900,50,33
9,10,2,2600,600,30,28


В таблице выше представлены результаты измерений производительности сетевых устройств №1 и №2. Они являются результатом серии экспериментов по проверке метода измерения производительности. Известно, что для всех устройств тестировали 1G интерфейс однородной смесью UDP-трафика

# Task 1

1.	Спрогнозировать значение UDP Throughput для сетевого устройства №3, имеющего параметры:

•	Тактовая частота процессора: 2 000 ГГц.

•	Утилизация процессора: 40%.

•	Температура процессора: 54 °С.

In [20]:
from sklearn.feature_selection import GenericUnivariateSelect, mutual_info_classif, SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LogisticRegression

# зафиксируем значение генератора случайных чисел для воспроизводимости 
SEED = 1

# Функции, которые в дальнейшем понадобятся
def plot_features_scores(model, data, target, column_names, model_type):
    '''Функция для визуализации важности признаков'''
    
    model.fit(data, target)
    
    if model_type == 'rf':
        (pd.DataFrame(data={'score': model['rf'].feature_importances_}, 
                      index=column_names).sort_values(by='score')
                                         .plot(kind='barh', grid=True,
                                               figsize=(6,6), legend=False));
    elif model_type == 'lr':
        (pd.DataFrame(data={'score': model['lr'].coef_[0]},
                      index=column_names).sort_values(by='score')
                                         .plot(kind='barh', grid=True,
                                               figsize=(6,6), legend=False));
        
    else:
        raise KeyError('Unknown model_type')

def grid_search(model, gs_params):
    '''Функция для подбора гиперпараметров с помощью перекрёстной проверки'''
     
    gs = GridSearchCV(estimator=model, param_grid=gs_params, refit=True,
                      scoring='roc_auc', n_jobs=-1, cv=skf, verbose=0)
    gs.fit(X, y)
    scores = [gs.cv_results_[f'split{i}_test_score'][gs.best_index_] for i in range(5)]
    print('scores = {}, \nmean score = {:.5f} +/- {:.5f} \
           \nbest params = {}'.format(scores,
                                      gs.cv_results_['mean_test_score'][gs.best_index_],
                                      gs.cv_results_['std_test_score'][gs.best_index_],
                                      gs.best_params_))
    return gs

In [22]:
# оставим только численые признаки
X = data.select_dtypes(exclude=['object']).copy()
# преобразуем целевую переменную
y = data['UDP_Throughput_Mb/sec'].map({'<=50K':0, '>50K':1}).values

In [23]:
X

,Measurement_number,network_device,CPU_clock_frequency_GHz,UDP_Throughput_Mb/sec,CPU_utilization_%,CPU_temperature_C
0,1,1,1500,600,50,38
1,2,1,1500,650,55,45
2,3,1,1500,700,60,52
3,4,1,1500,750,65,60
4,5,1,1500,900,70,67
5,6,2,2600,750,35,30
6,7,2,2600,800,40,31
7,8,2,2600,850,45,32
8,9,2,2600,900,50,33
9,10,2,2600,600,30,28


In [24]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   Measurement_number       10 non-null     int64
 1   network_device           10 non-null     int64
 2   CPU_clock_frequency_GHz  10 non-null     int64
 3   UDP_Throughput_Mb/sec    10 non-null     int64
 4   CPU_utilization_%        10 non-null     int64
 5   CPU_temperature_C        10 non-null     int64
dtypes: int64(6)
memory usage: 608.0 bytes


In [25]:
X.describe()

,Measurement_number,network_device,CPU_clock_frequency_GHz,UDP_Throughput_Mb/sec,CPU_utilization_%,CPU_temperature_C
count,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,5.50000,1.500000,2050.000000,750.000000,50.000000,41.600000
std,3.02765,0.527046,579.750904,113.038833,12.909944,13.801771
min,1.00000,1.000000,1500.000000,600.000000,30.000000,28.000000
25%,3.25000,1.000000,1500.000000,662.500000,41.250000,31.250000
50%,5.50000,1.500000,2050.000000,750.000000,50.000000,35.500000
75%,7.75000,2.000000,2600.000000,837.500000,58.750000,50.250000
max,10.00000,2.000000,2600.000000,900.000000,70.000000,67.000000
